In [2]:
##Testing function changes

In [3]:
import apache_beam

In [4]:
from datetime import datetime, timedelta
import numpy as np
import sigkernel
import torch
import weatherbench2
from multiprocessing import Pool, cpu_count
import os
import xarray as xr
import math
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion
from weatherbench2.evaluation import evaluate_in_memory
from weatherbench2 import config
from einops import rearrange
from itertools import product
import cython
import matplotlib.pyplot  as plt
import tqdm
#import Functions as fu
import line_profiler
import time

In [5]:
weights = np.array([0.07704437, 0.23039114, 0.38151911, 0.52897285, 0.67133229,
       0.80722643, 0.93534654, 1.05445875, 1.16341595, 1.26116882,
       1.34677594, 1.41941287, 1.47838008, 1.52310968, 1.55317091,
       1.56827425, 1.56827425, 1.55317091, 1.52310968, 1.47838008,
       1.41941287, 1.34677594, 1.26116882, 1.16341595, 1.05445875,
       0.93534654, 0.80722643, 0.67133229, 0.52897285, 0.38151911,
       0.23039114, 0.07704437])

weightslarger = np.array([0.00518318, 0.04146013, 0.08289184, 0.12426674, 0.16555647,
       0.20673274, 0.24776733, 0.28863211, 0.32929907, 0.36974035,
       0.40992823, 0.44983517, 0.48943381, 0.52869701, 0.56759787,
       0.60610973, 0.6442062 , 0.68186115, 0.7190488 , 0.75574364,
       0.79192053, 0.82755468, 0.86262167, 0.89709746, 0.93095842,
       0.96418135, 0.99674348, 1.02862249, 1.05979653, 1.09024424,
       1.11994476, 1.14887772, 1.17702329, 1.20436219, 1.23087569,
       1.2565456 , 1.28135434, 1.3052849 , 1.32832088, 1.35044651,
       1.3716466 , 1.39190663, 1.41121272, 1.42955164, 1.44691081,
       1.46327834, 1.47864302, 1.4929943 , 1.50632236, 1.51861807,
       1.52987299, 1.54007941, 1.54923034, 1.5573195 , 1.56434135,
       1.57029108, 1.57516462, 1.57895861, 1.58167047, 1.58329832,
       1.58384106, 1.58329832, 1.58167047, 1.57895861, 1.57516462,
       1.57029108, 1.56434135, 1.5573195 , 1.54923034, 1.54007941,
       1.52987299, 1.51861807, 1.50632236, 1.4929943 , 1.47864302,
       1.46327834, 1.44691081, 1.42955164, 1.41121272, 1.39190663,
       1.3716466 , 1.35044651, 1.32832088, 1.3052849 , 1.28135434,
       1.2565456 , 1.23087569, 1.20436219, 1.17702329, 1.14887772,
       1.11994476, 1.09024424, 1.05979653, 1.02862249, 0.99674348,
       0.96418135, 0.93095842, 0.89709746, 0.86262167, 0.82755468,
       0.79192053, 0.75574364, 0.7190488 , 0.68186115, 0.6442062 ,
       0.60610973, 0.56759787, 0.52869701, 0.48943381, 0.44983517,
       0.40992823, 0.36974035, 0.32929907, 0.28863211, 0.24776733,
       0.20673274, 0.16555647, 0.12426674, 0.08289184, 0.04146013,
       0.00518318])

In [6]:
#Given forecast of time T with predlag, and observations of time T + predlag
#Average each path of length k across all t.
#zero is either 1 or 0 
#latitude weighting


def pkparallel_lat_split1(lat_chunk, observations_chunk, forecasts_chunk, zero):
    """
    Function to compute results for a chunk of latitudes.
    """
    static_kernel = sigkernel.Linear_ID_Kernel()
    dyadic_order = 2
    signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

    time = forecasts_chunk.shape[0]
    lead = forecasts_chunk.shape[1]
    latlength = forecasts_chunk.shape[3]

    pkarray = np.zeros((latlength, time, lead, 3))

    for lat in range(latlength):
            for lag in range(2, lead + 1):

                batched_fors = torch.cat([
                torch.tensor(sigkernel.transform(np.expand_dims(forecasts_chunk[t,0:lag,:,lat], axis = 0), scale = 1, at = True, ll = False), dtype=torch.double)
                for t in range(time)
                ])
                batched_obs = torch.cat([
                    torch.tensor(sigkernel.transform(np.expand_dims(observations_chunk[2*t+zero:2*t+zero+lag,:,lat], axis = 0), scale = 1, at = True, ll = False), dtype=torch.double)
                    for t in range(time)
                ])

                K_Xy = signature_kernel.compute_kernel(batched_fors, batched_obs)
                K_XX = signature_kernel.compute_kernel(batched_fors, batched_fors)
                K_yy = signature_kernel.compute_kernel(batched_obs, batched_obs)

                pkarray[lat,:,lag-1,0] = K_Xy
                pkarray[lat,:,lag-1,1] = K_XX
                pkarray[lat,:,lag-1,2] = K_yy

    return pkarray

def pkparallel_lat_split(lat_chunk, observations_chunk, forecasts_chunk, zero):
    """
    Function to compute results for a chunk of latitudes.
    """
    static_kernel = sigkernel.Linear_ID_Kernel()
    dyadic_order = 2
    signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

    time = forecasts_chunk.shape[0]
    lead = forecasts_chunk.shape[1]
    latlength = forecasts_chunk.shape[3]

    pkarray = np.zeros((latlength, time, lead, 3))

    for lat in range(latlength):
        for t in range(time):
            for lag in range(2, lead + 1):
                fors = forecasts_chunk[t, 0:lag, :, lat]
                obs = observations_chunk[2 * t + zero:2 * t + zero + lag, :, lat]

                fors = np.expand_dims(fors, axis=0)  # Shape 1, lag, 64
                obs = np.expand_dims(obs, axis=0)  # Shape 1, lag, 64

                llobs = sigkernel.transform(obs, scale=1, at=True, ll=False)
                llfors = sigkernel.transform(fors, scale=1, at=True, ll=False)

                X = torch.tensor(llfors, dtype=torch.double, device = 'cuda')
                y = torch.tensor(llobs, dtype=torch.double, device = 'cuda')

                K_Xy = signature_kernel.compute_Gram(X, y, sym=False, max_batch=100)
                K_XX = signature_kernel.compute_Gram(X, X, sym=False, max_batch=100)
                K_yy = signature_kernel.compute_Gram(y, y, sym=False, max_batch=100)

                pkarray[lat, t, lag - 1, 0] = K_Xy
                pkarray[lat, t, lag - 1, 1] = K_XX
                pkarray[lat, t, lag - 1, 2] = K_yy

    return pkarray


def pkparallel(observations, forecasts, zero, batch_size=None):
    """
    Main function to parallelize computation across latitudes.
    """
    latlength = forecasts.shape[3]

    # Determine the number of processes (default to number of cores)
    num_cores = cpu_count()
    print(num_cores)
    batch_size = batch_size or (latlength // num_cores + (latlength % num_cores > 0))

    # Split data into chunks by latitude
    lat_chunks = [
        range(i, min(i + batch_size, latlength))
        for i in range(0, latlength, batch_size)
    ]
    print(lat_chunks)

    observations_chunks = [
        observations[:, :, lat_chunk]
        for lat_chunk in lat_chunks
    ]

    forecasts_chunks = [
        forecasts[:, :, :, lat_chunk]
        for lat_chunk in lat_chunks
    ]

    # Process chunks in parallel
    with Pool(processes=min(num_cores, len(lat_chunks))) as pool:
        results = pool.starmap(
            pkparallel_lat_split,
            [(lat_chunk, obs_chunk, for_chunk, zero)
             for lat_chunk, obs_chunk, for_chunk in zip(
                 lat_chunks, observations_chunks, forecasts_chunks
             )]
        )

    if latlength == 32:
        print("32")
        usedweights = weights
    if latlength == 121:
        print("121")
        usedweights = weightslarger

    # Combine results from all chunks
    #pkarraylat = np.sum(results, axis=0)
    pkarray = np.concatenate(results, axis=0) #Against lat chunks
    pkarraylat = np.sum(pkarray * usedweights[:, None, None, None], axis=0)
    pktime = np.mean(pkarraylat, axis=0)
    distance = pktime[:, 1] + pktime[:, 2] - 2 * pktime[:, 0]
    score = pktime[:, 1] - 2 * pktime[:, 0]

    return pkarray, pkarraylat, distance, score



def pkparallelcall(time_chunk, observations_chunk, forecasts_chunk, zero):
    """
    Function to compute results for a chunk of times.
    """
    static_kernel = sigkernel.Linear_ID_Kernel()   #Linear_ID_Kernel()  #RBFKernel(sigma=sigma)
    dyadic_order = 2
    signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)
    time = forecasts_chunk.shape[0]
    lead = forecasts_chunk.shape[1]
    latlength = forecasts_chunk.shape[3]
    
    pkarray = np.zeros((latlength,time,lead,3)) #,3
    #print(pkarray.shape)

    batch_size = latlength #or latlength
    # #lat_batches = [range(i, min(i + batch_size, latlength)) for i in range(0, latlength, batch_size)]

    for lag in range(2,lead+1): 
        #print(lag)
        for t in range(time):
            #print(t)
                # Stack tensors for all latitudes at once
            batched_fors = torch.cat([
                torch.tensor(sigkernel.transform(np.expand_dims(forecasts_chunk[t,0:lag,:,lat], axis = 0), scale = 1, at = True, ll = False), dtype=torch.double)
                for lat in range(latlength)
            ])
            batched_obs = torch.cat([
                torch.tensor(sigkernel.transform(np.expand_dims(observations_chunk[2*t+zero:2*t+zero+lag,:,lat], axis = 0), scale = 1, at = True, ll = False), dtype=torch.double)
                for lat in range(latlength)
            ])

            K_Xy = signature_kernel.compute_kernel(batched_fors, batched_obs, max_batch=batch_size)
            K_XX = signature_kernel.compute_kernel(batched_fors, batched_fors, max_batch=batch_size)
            K_yy = signature_kernel.compute_kernel(batched_obs, batched_obs, max_batch=batch_size)

            pkarray[:,t,lag-1,0] = K_Xy
            pkarray[:,t,lag-1,1] = K_XX
            pkarray[:,t,lag-1,2] = K_yy

    return pkarray



def pkparallel1(observations, forecasts, zero, batch_size=None):
    """
    Main function to parallelize computation across latitudes.
    """
    #Not currently set up for zero time forecasts
    timelength = forecasts.shape[0]
    lead = forecasts.shape[1]
    latlength = forecasts.shape[3]

    # Determine the number of processes (default to number of cores)
    num_cores = cpu_count()
    print(num_cores)
    batch_size = batch_size or (timelength // num_cores + (timelength % num_cores > 0))
    print(batch_size)
    
    # Split data into chunks by latitude
    time_chunks = [
        range(i, min(i + batch_size, timelength))
        for i in range(0, timelength, batch_size)
    ]
    print(time_chunks)

    observations_chunks = [
        observations[timechunk, :, :]
        for timechunk in time_chunks
    ]

    forecasts_chunks = [
        forecasts[timechunk, :, :, :]
        for timechunk in time_chunks
    ]

    # Process chunks in parallel
    with Pool(processes=min(num_cores, len(time_chunks))) as pool:
        results = pool.starmap(
            pkparallelcall,
            [(timechunk, obs_chunk, for_chunk, zero)
             for timechunk, obs_chunk, for_chunk in zip(
                 time_chunks, observations_chunks, forecasts_chunks
             )]
        )
    if latlength == 32:
        print("32")
        usedweights = weights
    if latlength == 121:
        print("121")
        usedweights = weightslarger

    # Combine results from all chunks
    #pkarraylat = np.sum(results, axis=0)
    pkarray = np.concatenate(results, axis=1) #Against time chunks 0 for lat chunks
    pkarraylat = np.sum(pkarray * usedweights[:, None, None, None], axis=0)
    pktime = np.mean(pkarraylat, axis=0)
    distance = pktime[:, 1] + pktime[:, 2] - 2 * pktime[:, 0]
    score = pktime[:, 1] - 2 * pktime[:, 0]

    return pkarray, pkarraylat, distance, score


def pkfulladjusted(observations,forecasts,zero):

        static_kernel = sigkernel.Linear_ID_Kernel()   #Linear_ID_Kernel()  #RBFKernel(sigma=sigma)
        dyadic_order = 2
        signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)
        time = forecasts.shape[0]
        lead = forecasts.shape[1]
        latlength = forecasts.shape[3]

        if latlength == 32:
             print("32")
             usedweights = weights
        if latlength == 121:
             print("121")
             usedweights = weightslarger


        #print(time,lead)
        
        pkarray = np.zeros((latlength,time,lead,3)) #,3
        #print(pkarray.shape)

        batch_size = latlength #or latlength
        # #lat_batches = [range(i, min(i + batch_size, latlength)) for i in range(0, latlength, batch_size)]

        for lag in range(2,lead+1): 
            print(lag)
            for t in range(time):
                #print(t)
                    # Stack tensors for all latitudes at once
                batched_fors = torch.cat([
                    torch.tensor(sigkernel.transform(np.expand_dims(forecasts[t,0:lag,:,lat], axis = 0), scale = 1, at = True, ll = False), dtype=torch.double)
                    for lat in range(latlength)
                ])
                batched_obs = torch.cat([
                    torch.tensor(sigkernel.transform(np.expand_dims(observations[2*t+zero:2*t+zero+lag,:,lat], axis = 0), scale = 1, at = True, ll = False), dtype=torch.double)
                    for lat in range(latlength)
                ])

                K_Xy = signature_kernel.compute_kernel(batched_fors, batched_obs, max_batch=batch_size)
                K_XX = signature_kernel.compute_kernel(batched_fors, batched_fors, max_batch=batch_size)
                K_yy = signature_kernel.compute_kernel(batched_obs, batched_obs, max_batch=batch_size)

                pkarray[:,t,lag-1,0] = K_Xy
                pkarray[:,t,lag-1,1] = K_XX
                pkarray[:,t,lag-1,2] = K_yy

        
        pkarraylat = np.sum(pkarray*usedweights[:,None,None,None],axis=0)
        pktime = np.mean(pkarraylat, axis=0)
        distance = pktime[:,1]+pktime[:,2]-2*pktime[:,0]
        score = pktime[:,1] - 2*pktime[:,0]

        return(pkarray,pkarraylat,distance,score)


##REtry with batching time, parrellelising lat
def workflowfullparallel(observations, forecasts, days, lag,zero):
    ob, fo = timecutting(observations,forecasts,days,lag+zero)
    ob = ob.values
    fo = fo[:,0:lag,:,:].values
    ob, fo = scalebyobsadjusted(ob,fo,fo.shape[2])
    pkarray,pkarraylat,distance,score = pkparallel(ob,fo,zero)

    return (pkarray,pkarraylat,distance,score)


def workflowfulladjusted(observations, forecasts, days, lag,zero):
    ob, fo = timecutting(observations,forecasts,days,lag+zero)
    ob = ob.values
    fo = fo[:,0:lag,:,:].values
    ob, fo = scalebyobsadjusted(ob,fo,fo.shape[2])
    pkarray,pkarraylat,distance,score = pkfulladjusted(ob,fo,zero)

    return (pkarray,pkarraylat,distance,score)

def timecutting(obsxarray,forsxarray, days, predtimedelta):
    startdate = datetime(2020,1,1)
    newdate = startdate + timedelta(days=days)
    obsnewdate = newdate + timedelta(hours = (predtimedelta-1)*6)
    
    return obsxarray['geopotential'].sel(level = 500, time = slice(startdate, obsnewdate)), forsxarray['geopotential'].sel(level = 500, time = slice(startdate, newdate)), 


def scalebyobsadjusted(observations,forecasts,shapeparam):
    mean = np.mean(observations)
    std = np.std(observations)
    scaledobs = (observations - mean)/(std*np.sqrt(shapeparam))
    scaledfors = (forecasts - mean)/(std*np.sqrt(shapeparam))

    return scaledobs, scaledfors

In [7]:
obs_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr'
observations = xr.open_zarr(obs_path)

forecast2 = xr.open_zarr('gs://weatherbench2/datasets/ifs_ens/2018-2022-64x32_equiangular_conservative_mean.zarr')
forecast2

<xarray.Dataset>
Dimensions:                   (time: 3652, prediction_timedelta: 61,
                               longitude: 64, latitude: 32, level: 3)
Coordinates:
  * latitude                  (latitude) float64 -87.19 -81.56 ... 81.56 87.19
  * level                     (level) int32 500 700 850
  * longitude                 (longitude) float64 0.0 5.625 ... 348.8 354.4
  * prediction_timedelta      (prediction_timedelta) timedelta64[ns] 00:00:00...
  * time                      (time) datetime64[ns] 2018-01-01 ... 2022-12-31...
Data variables: (12/15)
    10m_u_component_of_wind   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    10m_wind_speed            (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    2m_temperature            (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    geopotential              (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(4, 1, 3, 64, 32), meta=np.ndarray>
    mean_sea_level_pressure   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    ...                        ...
    total_precipitation       (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    total_precipitation_24hr  (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    total_precipitation_6hr   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(4, 1, 64, 32), meta=np.ndarray>
    u_component_of_wind       (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(4, 1, 3, 64, 32), meta=np.ndarray>
    v_component_of_wind       (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(4, 1, 3, 64, 32), meta=np.ndarray>
    wind_speed                (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(4, 1, 3, 64, 32), meta=np.ndarray>

In [8]:
obs_pathlarge = 'gs://weatherbench2/datasets/era5/1959-2023_01_10-6h-240x121_equiangular_with_poles_conservative.zarr'
observations240121 = xr.open_zarr(obs_pathlarge)
observations240121

<xarray.Dataset>
Dimensions:                                           (time: 93544,
                                                       longitude: 240,
                                                       latitude: 121, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -90....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/62)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    2m_dewpoint_temperature                           (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    above_ground                                      (time, level, longitude, latitude) float32 dask.array<chunksize=(8, 13, 240, 121), meta=np.ndarray>
    ...                                                ...
    volumetric_soil_water_layer_1                     (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    volumetric_soil_water_layer_2                     (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    volumetric_soil_water_layer_3                     (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    volumetric_soil_water_layer_4                     (time, longitude, latitude) float32 dask.array<chunksize=(8, 240, 121), meta=np.ndarray>
    vorticity                                         (time, level, longitude, latitude) float32 dask.array<chunksize=(8, 13, 240, 121), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 dask.array<chunksize=(8, 13, 240, 121), meta=np.ndarray>

In [9]:
forecast2larger = xr.open_zarr('gs://weatherbench2/datasets/ifs_ens/2018-2022-240x121_equiangular_with_poles_conservative_mean.zarr')
forecast2larger 

<xarray.Dataset>
Dimensions:                   (time: 3652, prediction_timedelta: 61,
                               longitude: 240, latitude: 121, level: 3)
Coordinates:
  * latitude                  (latitude) float64 -90.0 -88.5 -87.0 ... 88.5 90.0
  * level                     (level) int32 500 700 850
  * longitude                 (longitude) float64 0.0 1.5 3.0 ... 357.0 358.5
  * prediction_timedelta      (prediction_timedelta) timedelta64[ns] 00:00:00...
  * time                      (time) datetime64[ns] 2018-01-01 ... 2022-12-31...
Data variables: (12/15)
    10m_u_component_of_wind   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    10m_v_component_of_wind   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    10m_wind_speed            (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    2m_temperature            (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    geopotential              (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(1, 8, 3, 240, 121), meta=np.ndarray>
    mean_sea_level_pressure   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    ...                        ...
    total_precipitation       (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    total_precipitation_24hr  (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    total_precipitation_6hr   (time, prediction_timedelta, longitude, latitude) float32 dask.array<chunksize=(1, 8, 240, 121), meta=np.ndarray>
    u_component_of_wind       (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(1, 8, 3, 240, 121), meta=np.ndarray>
    v_component_of_wind       (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(1, 8, 3, 240, 121), meta=np.ndarray>
    wind_speed                (time, prediction_timedelta, level, longitude, latitude) float32 dask.array<chunksize=(1, 8, 3, 240, 121), meta=np.ndarray>

In [10]:
newpk2402, pkarraylat2402, distance2402,score2402 = workflowfulladjusted(observations240121, forecast2larger, days = 1, lag = 61, zero = 0)

121
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [11]:
newpk24021, pkarraylat24021, distance24021,score24021 = workflowfullparallel(observations240121, forecast2larger, days = 1, lag = 61, zero = 0)

12
[range(0, 11), range(11, 22), range(22, 33), range(33, 44), range(44, 55), range(55, 66), range(66, 77), range(77, 88), range(88, 99), range(99, 110), range(110, 121)]


: 

: 

In [19]:
distance = workflowfulladjusted(observations, forecast2, 2,61,0)

32
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [22]:
distance

array([0.00000000e+00, 3.02104863e-03, 5.85099162e-03, 7.31116702e-03,
       1.18141360e-02, 1.42991192e-02, 2.19586507e-02, 2.94111369e-02,
       4.15630754e-02, 5.44458802e-02, 7.48854381e-02, 9.43882855e-02,
       1.22598102e-01, 1.52838572e-01, 1.94489235e-01, 2.34742682e-01,
       2.90370724e-01, 3.46608539e-01, 4.19483785e-01, 4.86859925e-01,
       5.61840669e-01, 6.30331833e-01, 7.12479410e-01, 8.00033017e-01,
       9.10924494e-01, 1.01599473e+00, 1.13486299e+00, 1.24154637e+00,
       1.36069708e+00, 1.46778916e+00, 1.59119510e+00, 1.70352131e+00,
       1.83860080e+00, 1.96835005e+00, 2.11068919e+00, 2.26477797e+00,
       2.42923553e+00, 2.59853224e+00, 2.76589286e+00, 2.91595043e+00,
       3.06740399e+00, 3.19580758e+00, 3.34796602e+00, 3.48096795e+00,
       3.64703562e+00, 3.78142711e+00, 3.93717525e+00, 4.05184217e+00,
       4.18321646e+00, 4.29440599e+00, 4.43753116e+00, 4.58499644e+00,
       4.76582380e+00, 4.94503527e+00, 5.15583450e+00, 5.35903562e+00,
      

In [81]:
pkarray,pkarraylat,distance,score = workflowfulladjusted(observations, forecast2, 2,61,0)

32
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61


In [75]:
distance

array([0.00000000e+00, 3.02104863e-03, 5.85099162e-03, 7.31116702e-03,
       1.18141360e-02, 1.42991192e-02, 2.19586507e-02, 2.94111369e-02,
       4.15630754e-02, 5.44458802e-02, 7.48854381e-02, 9.43882855e-02,
       1.22598102e-01, 1.52838572e-01, 1.94489235e-01, 2.34742682e-01,
       2.90370724e-01, 3.46608539e-01, 4.19483785e-01, 4.86859925e-01,
       5.61840669e-01, 6.30331833e-01, 7.12479410e-01, 8.00033017e-01,
       9.10924494e-01, 1.01599473e+00, 1.13486299e+00, 1.24154637e+00,
       1.36069708e+00, 1.46778916e+00, 1.59119510e+00, 1.70352131e+00,
       1.83860080e+00, 1.96835005e+00, 2.11068919e+00, 2.26477797e+00,
       2.42923553e+00, 2.59853224e+00, 2.76589286e+00, 2.91595043e+00,
       3.06740399e+00, 3.19580758e+00, 3.34796602e+00, 3.48096795e+00,
       3.64703562e+00, 3.78142711e+00, 3.93717525e+00, 4.05184217e+00,
       4.18321646e+00, 4.29440599e+00, 4.43753116e+00, 4.58499644e+00,
       4.76582380e+00, 4.94503527e+00, 5.15583450e+00, 5.35903562e+00,
      

In [67]:
pkarray,pkarraylat,distance,score = workflowfulladjusted(observations, forecast2, 2,61,0)

32
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [68]:
distance

array([0.00000000e+00, 3.02104863e-03, 5.85099162e-03, 7.31116702e-03,
       1.18141360e-02, 1.42991192e-02, 2.19586507e-02, 2.94111369e-02,
       4.15630754e-02, 5.44458802e-02, 7.48854381e-02, 9.43882855e-02,
       1.22598102e-01, 1.52838572e-01, 1.94489235e-01, 2.34742682e-01,
       2.90370724e-01, 3.46608539e-01, 4.19483785e-01, 4.86859925e-01,
       5.61840669e-01, 6.30331833e-01, 7.12479410e-01, 8.00033017e-01,
       9.10924494e-01, 1.01599473e+00, 1.13486299e+00, 1.24154637e+00,
       1.36069708e+00, 1.46778916e+00, 1.59119510e+00, 1.70352131e+00,
       1.83860080e+00, 1.96835005e+00, 2.11068919e+00, 2.26477797e+00,
       2.42923553e+00, 2.59853224e+00, 2.76589286e+00, 2.91595043e+00,
       3.06740399e+00, 3.19580758e+00, 3.34796602e+00, 3.48096795e+00,
       3.64703562e+00, 3.78142711e+00, 3.93717525e+00, 4.05184217e+00,
       4.18321646e+00, 4.29440599e+00, 4.43753116e+00, 4.58499644e+00,
       4.76582380e+00, 4.94503527e+00, 5.15583450e+00, 5.35903562e+00,
      

In [47]:
static_kernel = sigkernel.RBFKernel(sigma=0.5)

# Specify dyadic order for PDE solver (int > 0, default 0, the higher the more accurate but slower)
dyadic_order = 1

# Specify maximum batch size of computation; if memory is a concern try reducing max_batch, default=100
max_batch = 100

# Initialize the corresponding signature kernel
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

# Synthetic data
batch, len_x, len_y, dim = 5, 10, 20, 2
X = torch.rand((batch,len_x,dim), dtype=torch.float64, device='cuda') # shape (batch,len_x,dim)
Y = torch.rand((batch,len_y,dim), dtype=torch.float64, device='cuda') # shape (batch,len_y,dim)
print(X.shape)
print(Y.shape)
# Compute signature kernel "batch-wise" (i.e. k(x_1,y_1),...,k(x_batch, y_batch))
K = signature_kernel.compute_kernel(X,Y,max_batch)

##Dist 
##        K_XX = self.compute_kernel(X, X, max_batch)
##        K_YY = self.compute_kernel(Y, Y, max_batch)
##        K_XY = self.compute_kernel(X, Y, max_batch)

##        return torch.mean(K_XX) + torch.mean(K_YY) - 2.*torch.mean(K_XY)

torch.Size([5, 10, 2])
torch.Size([5, 20, 2])


In [55]:
x=X
y=Y

In [65]:
np.expand_dims(x[0,:,:], axis = 0)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [64]:
signature_kernel.compute_kernel(np.expand_dims(x[0,:,:], axis = 0),np.expand_dims(y[0,:,:], axis = 0),max_batch)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [48]:
print(K)

tensor([0.3964, 0.6434, 1.2463, 1.3331, 0.3687], device='cuda:0',
       dtype=torch.float64)
